In [1]:
#pip install span_marker

In [2]:
from data_processing import get_subjects_by_Q_json, process_jsonl_and_json
from span_marker import SpanMarkerModel
from tqdm import tqdm
import time
import torch
import json
import numpy as np

spanishfile = '/Users/lolakovalski/Desktop/School/csci375/EAMT/data/spanish_w_labels.json'
italianfile = '/Users/lolakovalski/Desktop/School/csci375/EAMT/data/italian_w_labels.json'

model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd", clean_up_tokenization_spaces = True)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [3]:
with open(spanishfile, 'r') as jf:
    entity_data = [json.loads(line) for line in jf]

eng = entity_data[0]['source']
esp = entity_data[0]['target']
print(esp)

¿Qué actor protagonizó Titanic y nació en Los Ángeles, California?


In [4]:
entities = model.predict(eng)
spanish_entities = model.predict(esp)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
print(spanish_entities)
print(entities)

[{'span': 'Titanic', 'label': 'MEDIA', 'score': 0.9936309456825256, 'char_start_index': 23, 'char_end_index': 30}, {'span': 'Los Ángeles', 'label': 'LOC', 'score': 0.9999561309814453, 'char_start_index': 42, 'char_end_index': 53}, {'span': 'California', 'label': 'LOC', 'score': 0.999962329864502, 'char_start_index': 55, 'char_end_index': 65}]
[{'span': 'Titanic', 'label': 'VEHI', 'score': 0.5069388151168823, 'char_start_index': 28, 'char_end_index': 35}, {'span': 'Los Angeles', 'label': 'LOC', 'score': 0.999987006187439, 'char_start_index': 52, 'char_end_index': 63}, {'span': 'California', 'label': 'LOC', 'score': 0.9999725818634033, 'char_start_index': 65, 'char_end_index': 75}]


In [6]:
line = [entity_data[0]]
lstents = [ki['entities'] for ki in line]

##FROMSEMEVAL
keys = [ent for key in lstents for ent in key]
esents = [key[eskey]['es'] for eskey in keys for key in lstents]
enents = [key[enkey]['en'] for enkey in keys for key in lstents]
print(esents)
print(enents)

##FROM MODEL
print([entity['span'] for entity in spanish_entities])

['Los Ángeles']
['Los Angeles']
['Titanic', 'Los Ángeles', 'California']


In [7]:
test = np.array([entity['span'] for entity in spanish_entities], dtype = object)
print(test)

['Titanic' 'Los Ángeles' 'California']


In [8]:
true_labels = np.array([
    ["cat", "dog"],         # Observation 1
    ["mouse"],              # Observation 2
    ["elephant", "rabbit"]  # Observation 3
], dtype=object)

predicted_labels = np.array([
    ["dog", "cat"],         # Prediction for Observation 1
    ["mouse", "cat"],       # Prediction for Observation 2
    ["rabbit"]              # Prediction for Observation 3
], dtype=object)

# Initialize counters
TP, FP, FN = 0, 0, 0

# Loop through each observation
for true, pred in zip(true_labels, predicted_labels):
    true_set = set(true)
    pred_set = set(pred)
    
    TP += len(true_set & pred_set)  # Intersection: correctly predicted
    FP += len(pred_set - true_set)  # Predicted but not true
    FN += len(true_set - pred_set)  # True but not predicted

# Calculate metrics
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Precision: 0.80
Recall: 0.80
F1 Score: 0.80


In [9]:
import os
import json
with open(os.path.abspath('data/spanish_test.json'), 'r', encoding='utf-8') as jf:

    inputs = json.load(jf)
    #inputs = [json.loads(line) for line in jf]
print(inputs[0])

{'id': '25ed319e', 'source_locale': 'en', 'target_locale': 'es', 'source': 'What is the largest park in the capital city of the UK?', 'target': '¿Cuál es el parque más grande en la capital del Reino Unido?', 'entities': {'Q145': {'es': 'Reino Unido', 'en': 'United Kingdom'}}, 'from': 'mintaka'}


In [ ]:
# Import necessary functions
import os
import json
import random
from entity_masking import load_entities_mapping, mask_entities_with_spanmarker, ensure_pad_token
from mt_translation import translate_with_placeholders_m2m
from entity_reintegration import reintegrate_entities
from ner_processing import sanity_check, identify_named_entities
    
es_train = os.path.abspath('data/spanish_test.json')
it_train = os.path.abspath('data/italian_test.json')


print("Loading data... \n")

nerRes = {"es": None,"it": None}

# Step 2: Perform Named Entity Recognition (NER)
for language in ['es', 'it']:
    print("Identifying named entities {}... \n".format(language))
    file = es_train if language == 'es' else it_train
    with open(file, 'r', encoding='utf-8') as jf:
        inputs = json.load(jf)

    nerRes[language] = identify_named_entities(inputs, language, "test")
    input = {"entities_labeled": nerRes['target_entities_labeled'], "entities_predicted": nerRes['target_entities_predicted']}
""" 
    precision, recall, F1 = sanity_check(input)
    print("Accuracy of SpanMarker for Multilingual Named Entity Recognition {}\n".format(language))
    print("Precision: {}".format(precision))
    print("Recall: {}".format(recall))
    print("F1: {}".format(F1)) """


Loading data... 

Identifying named entities es... 



Processing target sentences: 100%|██████████| 299/299 [1:46:10<00:00, 21.31s/it]    


KeyError: 'target_entities_labeled'

In [11]:
nerRes["es"]

{'source': array(['What is the largest park in the capital city of the UK?',
        'Which Twilight books did not include the character Renesmee?',
        'What is the third smallest country in Africa?',
        'Where was the first all-woman city council in the United States?',
        'What is the population of the most populous country in South America?',
        'How many Supreme Leaders of North Korea have there been?',
        'Who became the president of Germany on 1 July 1994?',
        'What is the largest city in England by population?',
        'Who is the oldest pitcher to throw a no-hitter?',
        "Which was written first, Fear and Loathing in Las Vegas or Fear and Loathing on the Campaign Trail '72?",
        'Which main character in Twilight was not a vampire or werewolf?',
        'Which war lasted longer, Second Seminole War or World War II?',
        'Who was the first pope?',
        'What is the last book in the Lord of the Rings series? 4?',
        "Who is Mi

In [21]:
input = {"entities_labeled": nerRes['es']['target_entities_labeled'], "entities_predicted": nerRes['es']['target_entities_predicted']}

entities_labeled = input['entities_labeled']
entites_predicted = input['entities_predicted']
TP, FP, FN = 0, 0, 0

# Loop through each observation
for true, pred in zip(entities_labeled, entites_predicted):
    true_set = set(true)
    pred_set = set(pred)
    
    TP += len(true_set & pred_set)  # Intersection: correctly predicted
    FP += len(pred_set - true_set)  # Predicted but not true
    FN += len(true_set - pred_set)  # True but not predicted

# Calculate metrics
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print("Accuracy of SpanMarker for Multilingual Named Entity Recognition {}\n".format(language))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F1: {}".format(FN))

Accuracy of SpanMarker for Multilingual Named Entity Recognition es

Precision: 0.0
Recall: 0.0
F1: 2721


In [19]:
input['entities_labeled']

array(['Reino Unido', 'Crepúsculo', 'África', 'Estados Unidos',
       'América del Sur', 'Corea del Norte', 'Alemania', 'Inglaterra',
       'lanzador', 'Miedo y asco en Las Vegas', 'Crepúsculo',
       'Segunda Guerra Mundial', 'Segunda guerra semínola', 'papa',
       'El Señor de los Anillos', 'Partido Demócrata', 'Stephenie Meyer',
       'Crepúsculo', 'Canadá', 'Estados Unidos', 'Inglaterra',
       'Estados Unidos', 'República Popular China', 'Rusia',
       'Monte Rainier', 'Estados Unidos', 'Illinois', 'Sudáfrica',
       'Proclamación de Emancipación', 'América del Sur', 'África',
       'Estados Unidos', 'Universidad de Pensilvania', 'París', 'Francia',
       'Estados Unidos', 'Segunda Guerra Mundial', 'Matar un ruiseñor',
       'Nueva Zelanda', 'México', 'Everest', 'Brasil', 'Estados Unidos',
       'Universo cinematográfico de Marvel', 'Estados Unidos',
       'Corea del Sur', 'El Señor de los Anillos', 'Europa',
       'Casa Blanca', 'Imperio mongol', 'Japón', 'guerra d

In [22]:
input['entities_predicted']

array([list(['Reino Unido']), list(['Crepúsculo', 'Renesmee']),
       list(['África']), list(['Estados Unidos']),
       list(['América del Sur']), list(['Corea del Norte']),
       list(['Alemania']), list(['Inglaterra']), list([]),
       list(['Las Vegas']), list(['Crepúsculo']),
       list(['Segunda Guerra Seminola', 'Segunda Guerra Mundial']),
       list(['¿Quién fue el primer Papa?']),
       list(['El Señor de los Anillos']),
       list(['Mitch McConnell', 'Senado de Kentucky']),
       list(['Stephanie Meyer', 'Edward Cullen']),
       list(['Canadá', 'Estados Unidos']), list(['Inglaterra']),
       list(['Estados Unidos']), list([]), list(['Rusia']),
       list(['Monte Rainier']), list(['Estados Unidos', 'Copa Mundial']),
       list(['Missouri', 'Ilinois']), list(['Sudáfrica']),
       list(['Illinois', 'Proclamación de Emancipación']),
       list(['América del Sur']), list(['África']),
       list(['Estados Unidos']), list(['Universidad de Pensilvania']),
       list([

In [25]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Specify the model name
model_name = "facebook/m2m100_418M"  # or "facebook/m2m100_1.2B" for a larger model

# Load the tokenizer and model
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

# Define the source and target languages
source_language = "en"  # English
target_language = "it"  

# Set the tokenizer's source language
tokenizer.src_lang = source_language

# Input text to translate
text = "Who is supposed to have founded Rome?"

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")

# Generate the translation
generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_language])

# Decode the translated text
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
print("Translation:", translation)

Translation: Chi avrebbe fondato Roma?


In [3]:
import json
from nltk.translate.bleu_score import sentence_bleu

def load_json_data(json_file_path):
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            return json.load(f)  # Parses a JSON array
    except json.JSONDecodeError:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            return [json.loads(line.strip()) for line in f]

def calculate_bleu_scores(json_file_path):
    json_data = load_json_data(json_file_path)
    results = []
    bleu_1_total, bleu_2_total, bleu_3_total, bleu_4_total = 0, 0, 0, 0
    num_objects = len(json_data)

    for entry in json_data:
        target = entry["target"].split()
        translation = entry["translation"].split()

        # Calculate BLEU scores
        bleu_1 = sentence_bleu([target], translation, weights=(1, 0, 0, 0))
        bleu_2 = sentence_bleu([target], translation, weights=(0.5, 0.5, 0, 0))
        bleu_3 = sentence_bleu([target], translation, weights=(0.33, 0.33, 0.33, 0))
        bleu_4 = sentence_bleu([target], translation, weights=(0.25, 0.25, 0.25, 0.25))

        # Update totals
        bleu_1_total += bleu_1
        bleu_2_total += bleu_2
        bleu_3_total += bleu_3
        bleu_4_total += bleu_4

        # Store individual results
        results.append({
            "id": entry["id"],
            "bleu_1": bleu_1,
            "bleu_2": bleu_2,
            "bleu_3": bleu_3,
            "bleu_4": bleu_4
        })

    # Calculate averages
    averages = {
        "average_bleu_1": bleu_1_total / num_objects if num_objects > 0 else 0,
        "average_bleu_2": bleu_2_total / num_objects if num_objects > 0 else 0,
        "average_bleu_3": bleu_3_total / num_objects if num_objects > 0 else 0,
        "average_bleu_4": bleu_4_total / num_objects if num_objects > 0 else 0
    }

    return averages, results

# Example usage
json_file_path = '/Users/lolakovalski/Desktop/School/csci375/EAMT/data/translated_spanish_test_new.json'

average_bleu_scores, individual_scores = calculate_bleu_scores(json_file_path)

# Print average BLEU scores
print("Average BLEU Scores:", average_bleu_scores)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)